# Compare number of rows from single day sample to longer time average 

If we aggregate across time (single dates rolled into quarterly and annual averages), how many rows get reduced? Are we capturing the same number of groupings?

If we aggregate across space within operator (shapes-stop_seq -> route/dir/stop_pair), how many rows get reduced?

Scripts:
* [quarterly and annual averages](https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/scripts/quarter_year_averages.py)

Past notebooks:
* [segment stability notebook](https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/39_segment_stability.ipynb)
* [weekly average speeds](https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/26_weekly_avg_speeds.ipynb)
* [stop combinations notebook](https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/16_stop_combos_for_segments.ipynb)

In [1]:
import polars as pl
import pandas as pd

from shared_utils import dask_utils, rt_dates
from segment_speed_utils.project_vars import GTFS_DATA_DICT, SEGMENT_GCS

In [2]:
# compare Mar-Dec 2023 dates + Jan-Oct 2024 dates 
# (shape speeds are not produced after Oct)
analysis_date_list = rt_dates.y2023_dates + rt_dates.y2024_dates[:-2]

SPEED_FILE = GTFS_DATA_DICT.stop_segments.route_dir_single_segment
SPEED_SHAPE_FILE = "rollup_singleday/speeds_shape_stop_segments"

route_dir_cols = ["route_id", "direction_id", "stop_pair"]
shape_cols = ["shape_array_key", "stop_pair"]

# including caltrans_district will separate some dates for an operator
# since the caltrans_district value changed at some point
operator_cols = ["name"] 
other_metrics = ["n_trips"]

In [3]:
def concatenated_df(
    file_name: str,
    analysis_date_list: list,
    columns: list,
    time_period: str = "all_day",
) -> pd.DataFrame:
    """
    Import all the dates and compare some metrics on
    number of groupings, rows, etc for each operator.
    """
    paths = [f"{SEGMENT_GCS}{file_name}" for d in analysis_date_list]

    df = dask_utils.get_ddf(
        paths, 
        analysis_date_list, 
        data_type = "df",
        get_pandas = True,
        add_date = True,
        columns = columns,
        filters = [[("time_period", "==", time_period)]]
    )
    
    return df

In [4]:
def operator_groupings_by_date(
    df: pd.DataFrame,
    group_cols: list
) -> pd.DataFrame:
    """
    Count number of groups for each operator by date.
    Also count total trips covered.
    Since a group is a "segment", n_trips is counting 
    trips covered by that segment.
    """
    df2 = (
        df
        .groupby(group_cols, group_keys=False, dropna=False)
        .agg({
            "n_trips": "sum",
            "stop_pair": "count"
        })
        .reset_index()
        .rename(columns = {"stop_pair": "n_groups"})
    )

    return df2

In [5]:
route_dir_df = concatenated_df(
    SPEED_FILE,
    analysis_date_list,
    columns = operator_cols + route_dir_cols + other_metrics
).pipe(operator_groupings_by_date, operator_cols + ["service_date"])

shape_df = concatenated_df(
    SPEED_SHAPE_FILE,
    analysis_date_list,
    columns = operator_cols + shape_cols + other_metrics
).pipe(operator_groupings_by_date, operator_cols + ["service_date"])

In [6]:
df = pd.merge(
    route_dir_df.rename(columns = {
        "n_trips": "route_dir_n_trips",
        "n_groups": "route_dir_groups"
    }),
    shape_df.rename(columns = {
        "n_trips": "shape_n_trips",
        "n_groups": "shape_groups"
    }),
    on = operator_cols + ["service_date"],
    how = "inner",
)

In [7]:
# Calculate differences
df = df.assign(
    n_groups_diff = df.route_dir_groups - df.shape_groups,
    n_trips_diff = df.route_dir_n_trips - df.shape_n_trips,
)

In [8]:
def transform_for_nanoplot(
    df: pd.DataFrame,
    group_cols: list = ["name"]
):
    
    other_cols = [c for c in df.columns if c not in group_cols and c != "service_date"]
    df2 = (
        df
        .sort_values(group_cols + ["service_date"])
        .groupby(group_cols)
        .agg({
            **{c: lambda x: list(x) for c in other_cols}
        })
        .reset_index()
    )
    
    return df2

In [9]:
df_flat = transform_for_nanoplot(df)

In [10]:
from great_tables import GT, nanoplot_options

df_pl = pl.from_pandas(df_flat)

In [11]:
nano_options=nanoplot_options(
    data_bar_stroke_color="dodgerblue",
    data_bar_stroke_width=2,
    data_bar_fill_color="dodgerblue",
    data_bar_negative_stroke_color="coral",
    data_bar_negative_stroke_width=1,
    data_bar_negative_fill_color="coral",
    reference_line_color="darkgray",
    reference_area_fill_color="green",
    vertical_guide_stroke_color="dodgerblue",
)

In [12]:
def make_plot(df):
    """
    Make table.
    """
    table = (GT(df)
     .fmt_nanoplot(
        columns = "n_groups_diff", 
        reference_line = 0,
        plot_type = "bar",
        options = nano_options
    ).fmt_nanoplot(
         columns = "n_trips_diff", 
         reference_line = 0,
         plot_type = "bar", 
         options = nano_options
     ).cols_label(
         name = "Operator",
         n_groups_diff = "# Groups (Route-Dir - Shape)",
         n_trips_diff = "# Trips (Route-Dir - Shape)",
     ).cols_hide(
         ["route_dir_n_trips", "route_dir_groups",
          "shape_n_trips", "shape_groups"]
     ).tab_header(
         title = "Aggregation Groups by Date",
         subtitle = "route-direction-stop_pair vs shape-stop_pair"
     )
    )

    return table

In [13]:
make_plot(df_pl)

GT(_tbl_data=shape: (92, 7)
┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ name         ┆ route_dir_n ┆ route_dir_g ┆ shape_n_tri ┆ shape_group ┆ n_groups_di ┆ n_trips_dif │
│ ---          ┆ _trips      ┆ roups       ┆ ps          ┆ s           ┆ ff          ┆ f           │
│ str          ┆ ---         ┆ ---         ┆ ---         ┆ ---         ┆ ---         ┆ ---         │
│              ┆ list[i64]   ┆ list[i64]   ┆ list[i64]   ┆ list[i64]   ┆ list[i64]   ┆ list[i64]   │
╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ Antelope     ┆ [14801,     ┆ [999, 974,  ┆ [14800,     ┆ [1031, 997, ┆ [-32, -23,  ┆ [1, 1, … 1] │
│ Valley       ┆ 14398, …    ┆ … 1047]     ┆ 14397, …    ┆ … 1061]     ┆ … -14]      ┆             │
│ Transit      ┆ 14869]      ┆             ┆ 14868]      ┆             ┆             ┆             │
│ Author…      ┆             ┆             ┆             ┆             ┆             ┆             │
│ Avalon       ┆ [17, 161, … ┆ [12, 14, …  ┆ [17, 161, … ┆ [12, 14, …  ┆ [0, 0, … 0] ┆ [0, 0, … 7] │
│ Schedule     ┆ 149]        ┆ 14]         ┆ 142]        ┆ 14]         ┆             ┆             │
│ B-Line       ┆ [6547,      ┆ [690, 679,  ┆ [6547,      ┆ [894, 870,  ┆ [-204,      ┆ [0, 0, … 0] │
│ Schedule     ┆ 6540, …     ┆ … 682]      ┆ 6540, …     ┆ … 872]      ┆ -191, …     ┆             │
│              ┆ 6533]       ┆             ┆ 6533]       ┆             ┆ -190]       ┆             │
│ Banning Pass ┆ [194, 184,  ┆ [20, 20, …  ┆ [194, 194,  ┆ [36, 33, …  ┆ [-16, -13,  ┆ [0, -10, …  │
│ Schedule     ┆ … 146]      ┆ 20]         ┆ … 146]      ┆ 32]         ┆ … -12]      ┆ 0]          │
│ Bay Area 511 ┆ [208661,    ┆ [8420,      ┆ [208661,    ┆ [10102,     ┆ [-1682,     ┆ [0, 0, … 0] │
│ AC Transit   ┆ 204192, …   ┆ 7804, …     ┆ 204192, …   ┆ 9413, …     ┆ -1609, …    ┆             │
│ Schedu…      ┆ 197122]     ┆ 8628]       ┆ 197122]     ┆ 10351]      ┆ -1723]      ┆             │
│ …            ┆ …           ┆ …           ┆ …           ┆ …           ┆ …           ┆ …           │
│ Victor       ┆ [14531,     ┆ [1131,      ┆ [14531,     ┆ [1260,      ┆ [-129,      ┆ [0, 0, … 0] │
│ Valley GMV   ┆ 13685, …    ┆ 1109, …     ┆ 13685, …    ┆ 1232, …     ┆ -123, …     ┆             │
│ Schedule     ┆ 14344]      ┆ 1106]       ┆ 14344]      ┆ 1192]       ┆ -86]        ┆             │
│ Visalia      ┆ [4928,      ┆ [427, 442,  ┆ [4928,      ┆ [427, 442,  ┆ [0, 0, … 0] ┆ [0, 0, … 0] │
│ Schedule     ┆ 9170, …     ┆ … 443]      ┆ 9170, …     ┆ … 443]      ┆             ┆             │
│              ┆ 8809]       ┆             ┆ 8809]       ┆             ┆             ┆             │
│ WeHo         ┆ [146, 154,  ┆ [14, 14, …  ┆ [146, 154,  ┆ [14, 14, …  ┆ [0, 0, … 0] ┆ [0, 0, … 0] │
│ Schedule     ┆ … 587]      ┆ 84]         ┆ … 587]      ┆ 84]         ┆             ┆             │
│ Yolobus      ┆ [4885,      ┆ [495, 473,  ┆ [4885,      ┆ [495, 473,  ┆ [0, 0, … 0] ┆ [0, 0, … 0] │
│ Schedule     ┆ 4698, …     ┆ … 495]      ┆ 4698, …     ┆ … 495]      ┆             ┆             │
│              ┆ 4569]       ┆             ┆ 4569]       ┆             ┆             ┆             │
│ eTrans       ┆ [81, 54, …  ┆ [39, 35, …  ┆ [81, 54, …  ┆ [39, 35, …  ┆ [0, 0, … 0] ┆ [0, 0, … 0] │
│ Schedule     ┆ 68]         ┆ 32]         ┆ 68]         ┆ 32]         ┆             ┆             │
└──────────────┴─────────────┴─────────────┴─────────────┴─────────────┴─────────────┴─────────────┘, _body=<great_tables._gt_data.Body object at 0x793b949bce50>, _boxhead=Boxhead([ColInfo(var='name', type=<ColInfoTypeEnum.default: 1>, column_label='Operator', column_align='left', column_width=None), ColInfo(var='route_dir_n_trips', type=<ColInfoTypeEnum.hidden: 4>, column_label='route_dir_n_trips', column_align='center', column_width=None), ColInfo(var='route_dir_groups', type=<ColInfoTypeEnum.hidden: 4>, column_label='route_dir_gr

### Quick sanity check for inner merge fails

Only 1 operator is in common, and that date had very sparse observations.

In [14]:
merged = pd.merge(
    route_dir_df.rename(columns = {
        "n_trips": "route_dir_n_trips",
        "n_groups": "route_dir_groups"
    }),
    shape_df.rename(columns = {
        "n_trips": "shape_n_trips",
        "n_groups": "shape_groups"
    }),
    on = operator_cols + ["service_date"],
    how = "outer",
    indicator=True
)

In [15]:
merged._merge.value_counts()

both          1475
right_only     118
left_only        4
Name: _merge, dtype: int64

In [16]:
merged[merged._merge=="left_only"][operator_cols].drop_duplicates()

,name
38,B-Line Schedule
73,Bay Area 511 ACE Schedule
909,Mendocino Schedule
1102,Redwood Coast Schedule


In [17]:
merged[merged._merge=="right_only"][operator_cols].drop_duplicates()

,name
1479,Anaheim Resort Schedule
1498,Anaheim Resort Schedule v2
1500,Arcadia Schedule
1510,Bay Area 511 ACE Schedule
1511,BruinBus Schedule
1515,Irvine CONNECT Schedule
1519,LAX Flyaway Bus Schedule
1521,LAX Shuttles Schedule
1523,MV Shuttle Schedule
1525,Montebello Schedule


In [18]:
merged[(merged._merge=="left_only") & (merged.name=="Bay Area 511 ACE Schedule")]

,name,service_date,route_dir_n_trips,route_dir_groups,shape_n_trips,shape_groups,_merge
73,Bay Area 511 ACE Schedule,2024-09-18,2.0,2.0,NaN,NaN,left_only


In [19]:
merged[(merged._merge=="right_only") & (merged.name=="Bay Area 511 ACE Schedule")]

,name,service_date,route_dir_n_trips,route_dir_groups,shape_n_trips,shape_groups,_merge
1510,Bay Area 511 ACE Schedule,2024-07-17,NaN,NaN,1.0,1.0,right_only
